# Database creation
Since I started collecting music I was concerned about finding the stuff I had. In the beginning my memory was enough, but as albums, singles & EPs, bootlegs, and compilations started piling up, I had to look for a way to keep track of what I had and where it was. I started a plain .txt file with Artist - Album (year) - Storage as an easy way to search for things I wanted to listen to, and this file gradually grew over the years until I stopped regularly downlading music and started relying more on streaming services around mid-2015.

Here I tried to create a database that was easier to query with the goal of finding and rediscovering "old" music in my library. Having built that .txt by hand means there's probably a lot of typos and other inconsistencies so it will not be a perfect solution, but it's good enough for my purpose: to revisit old favourites.

In [1]:
import sqlite3
import os
import pandas as pd
import re
import matplotlib.pyplot as plt

In [2]:
# read .txt
def load_library(filename):
    """
    Load the original .txt file with Artist - Album (year) - Storage sctructure.
    Note: I did some checks to find out what encoding it used because it was not UTF-8.
    """
    with open(filename, "r", encoding='ISO-8859-1') as file:
        raw_data = file.read()
    
    return raw_data

def raw_to_df(raw_data):
    """
    Put the raw text into a dataframe, and only keep albums for now.
    """
    # only do albums, so parse until there's a double newline
    sections = raw_data.split('\n\n')

    # initialise data
    data = []

    # define pattern to match each line (i.e. each album: Artist - Album (year) - storage)
    # inconsistencies: some albums have comments like (Deluxe Edition), some are missing storage info
    pattern = r'^(.*?) - (.*?) \((\d{4})\)(?:\s*\((.*?)\))?(?: - (.*?))?$'

    lines = sections[0].split('\n')

    n_skipped = 0
    for line in lines:
        match = re.match(pattern, line.strip())
        if match:
            artist, album, year, comment, storage = match.groups()
            year = int(year) if year else None
            comment = comment.strip() if comment else None
            storage = storage.strip() if storage else None
            data.append({'artist_name': artist,
                         'album_name': album,
                         'year': int(year),
                         'comment': comment,
                         'storage': storage})
        else:
            n_skipped += 1
            print(f"Skipping line (format mismatch): {line}")
    print(f"Total lines skipped: {n_skipped}")

    # put in dataframe
    df = pd.DataFrame(data)
    
    return df

In [3]:
# load data
raw_data = load_library("Discos.txt")
df = raw_to_df(raw_data)


Total lines skipped: 0


In [4]:
# filter df to keep artist, album name, and year. I don't care about comments and storage for this.
df_filtered = df[["artist_name", "album_name", "year"]]

# Create SQLite db
This database will have tables that correspond to:
1. Artists: with info I find useful for future queries.
2. Albums: basically follows the structure of the original .txt but with added fields I find useful.


In [2]:
def connect_to_db(db_directory, db_name = 'music_library.db'):
    """
    Connect to the database.
    """
    # check if folder exists
    if not os.path.exists(db_directory):
        os.makedirs(db_directory)
    
    # create full path for the db
    db_path = os.path.join(db_directory, db_name)
    
    # connect to SQLite db
    conn = sqlite3.connect(db_path)
    return conn

In [3]:
# connect to db and create tables
save_dir = '/Users/carolinashimabukuro/projects/my-oldies-radio/'

conn = connect_to_db(save_dir)
cursor = conn.cursor()

create_artists_table = """
CREATE TABLE IF NOT EXISTS artists (
    artist_name TEXT PRIMARY KEY,
    mb_artist_id TEXT UNIQUE,
    country TEXT,
    city TEXT,
    genre TEXT,
    lastfm_url TEXT,
    spotify_url TEXT
);
"""

create_albums_table = """
CREATE TABLE IF NOT EXISTS albums (
    album_name TEXT NOT NULL,
    mb_album_id TEXT,
    mb_release_group_id TEXT,
    artist_name TEXT NOT NULL,
    mb_artist_id TEXT,
    year INTEGER,
    label TEXT,
    FOREIGN KEY (mb_artist_id) REFERENCES artists (mb_artist_id)
);
"""

cursor.execute(create_artists_table)
cursor.execute(create_albums_table)

conn.commit()
conn.close()

In [35]:
# let's put the df info in the albums table
conn = connect_to_db(save_dir)
cursor = conn.cursor()

insert_album_sql = """
INSERT INTO albums (artist_name, album_name, year)
VALUES (?, ?, ?);
"""

for _, row in df_filtered.iterrows():
    cursor.execute(insert_album_sql, (row["artist_name"], row["album_name"], row["year"]))

conn.commit()
conn.close()

# Populate tables
First we will start with the albums table. This is because it's possible that querying an artist by its name will not retrieve the correct MusicBrainz ID due to ambiguity or low popularity, e.g. Jonny (Euros Childs + Norman Blake) are not popular enough in MusicBrainz or Spotify, so if I select the first artist that kind of matches "Jonny" I get the wrong artist. The probability of getting the wrong artist decreases if I first query an album, because I have information about the artist and year as well. And from there I can get other info including the MusicBrainz ID of the album artist.


In [29]:
import musicbrainzngs
import time
import requests
from dotenv import load_dotenv

# MB user agent
musicbrainzngs.set_useragent(
    "my-oldies-radio",
    "0.1",
)

# Discogs token
load_dotenv()
DISCOGS_KEY = os.getenv('DISCOGS_KEY')

def fetch_album_info(artist_name, album_name):
    """Search for an album on MusicBrainz based on artist and album title."""
    
    query = f"sortname:{artist_name} AND name:{artist_name} AND release:{album_name}"
    album_info = musicbrainzngs.search_releases(query, strict=True)
    
    return album_info

def get_label(album_name, artist_name, DISCOGS_KEY):
    """Get label info from Discogs."""
    
    url = "https://api.discogs.com/database/search"
    headers = {
        "Authorization": "Discogs token="
    }
    params = {
        "q": album_name,
        "artist": artist_name,
    }
    response = requests.get(url, headers=headers, params=params)
    resp_json = response.json()
    if not resp_json.get("results", []):
        label = "not available"
    else:
        label = resp_json.get("results", [])[0]["label"][0]
    return label

def check_artist_exists_id(cursor, mb_artist_id):
    """
    Check if an artist already exists in the artist table.
    """
    
    cursor.execute(
        "SELECT 1 FROM artists WHERE mb_artist_id = ?",
        (mb_artist_id,)
    )
    
    return cursor.fetchone() is not None

def check_artist_exists_name(cursor, artist_name):
    """
    Check if an artist already exists in the artist table.
    """
    
    cursor.execute(
        "SELECT 1 FROM artists WHERE artist_name = ?",
        (artist_name,)
    )
    
    return cursor.fetchone() is not None

def fetch_artist_info(mb_artist_id):
    """
    Browse artist on MusicBrainz based on unique MBID to get artist info.
    """
    
    artist_info = musicbrainzngs.get_artist_by_id(mb_artist_id, includes=["area-rels", "url-rels", "tags"])
    
    return artist_info

def update_artists_table(cursor, artist_name, mb_artist_id, country, city, genre, lastfm_url, spotify_url): #artist_name, mb_artist_id, country, city, genre, lastfm_url, spotify_url
    """
    Updates the artists table by adding their MusicBrainz ID.
    """
    
    cursor.execute(
        """
        INSERT OR IGNORE INTO artists (artist_name, mb_artist_id, country, city, genre, lastfm_url, spotify_url)
        VALUES (?,?,?,?,?,?,?)
        """,
        (artist_name, mb_artist_id, country, city, genre, lastfm_url, spotify_url)
    )


def update_albums_table(cursor, album_name, mb_album_id, mb_release_group_id, artist_name, mb_artist_id, year, label):
    """
    Updates the albums table by adding the info extracted from MusicBrainz and Discogs.
    """
    
    cursor.execute(
        """
        UPDATE albums
        SET mb_album_id = ?,
            mb_release_group_id = ?,
            mb_artist_id = ?,
            label = ?
        WHERE album_name = ? AND artist_name = ? AND (mb_album_id IS NULL OR mb_album_id = '')
        """,
        (mb_album_id,
         mb_release_group_id,
         mb_artist_id,
         label,
         album_name,
         artist_name)
    )

def get_top_genre(artist_info):
    """
    Get tags associated with an artist, then sort them and pick the top tag. These are up and downvoted
    by users so might not be accurate.
    """
    # turn tag list into df
    genredf = pd.DataFrame(artist_info["artist"]["tag-list"], columns=["count","name"])
    # convert count strings into int
    genredf['count'] = genredf['count'].astype(int)
    # sort by value
    genredf.sort_values(by="count", ascending=False, ignore_index=True, inplace=True)
    # select top tag name
    top_genre = genredf.iloc[0]["name"]
    
    return top_genre

def get_urls(artist_info):
    for link in artist_info["artist"]["url-relation-list"]:
        if "open.spotify" in link["target"]:
            spotify = link["target"]
        else:
            spotify = None
    return spotify

def process_batch(db_path, batch_size=50, delay=1):
    """Search for album info and extract info to populate tables."""
    
    # connect to db
    conn = connect_to_db(save_dir)
    cursor = conn.cursor()
    
    cursor.execute("SELECT artist_name, album_name, year, rowid FROM albums WHERE mb_album_id IS NULL")
    rows = cursor.fetchall()
    
    try:
        # iterate over rows to search albums
        for albumrow in range(0, len(rows), batch_size):
            batch = rows[albumrow:albumrow + batch_size]
            for artist_name, album_name, year, rowid in batch:
                print(f"Row {rowid}: {artist_name} - {album_name} ({year})")

                album_info = fetch_album_info(artist_name, album_name)
                if album_info["release-list"]: # some might be missing
                    mb_artist_id     = album_info["release-list"][0]["artist-credit"][0]["artist"]["id"]
                    mb_album_id      = album_info["release-list"][0]["id"]
                    label = get_label(album_name, artist_name)
                    mb_release_group_id = album_info["release-list"][0]["release-group"]["id"]

                    # update albums table
                    update_albums_table(cursor, album_name, mb_album_id, mb_release_group_id, artist_name, mb_artist_id, year, label)

                    artist_check = check_artist_exists_id(cursor, mb_artist_id)
                    if artist_check: #and artist_check[0] is not None:
                        continue
                    else:
                        # get artist data
                        artist_info = fetch_artist_info(mb_artist_id)
                        if "country" in artist_info["artist"]:
                            country = artist_info["artist"]["country"]
                        else: None
                        if "begin-area" in artist_info["artist"]:
                            city = artist_info["artist"]["begin-area"]["name"]
                        else: None
                        if "tag-list" in artist_info["artist"] and artist_info["artist"]["tag-list"]:
                            genre = get_top_genre(artist_info)
                        else: None
                        lastfm_url = f"https://www.last.fm/music/{artist_name}"
                        if "url-relation-list" in artist_info["artist"]:
                            spotify_url = get_urls(artist_info)

                        # update artists table
                        update_artists_table(cursor, artist_name, mb_artist_id, country, city, genre, lastfm_url, spotify_url)

                else:
                    print(f"ERROR :(")

                    # update anyway
                    mb_artist_id = 'not available'
                    mb_album_id = 'not available'
                    mb_release_group_id = 'not available'
                    label = 'not available'
                    # update albums table
                    update_albums_table(cursor, album_name, mb_album_id, mb_release_group_id, artist_name, mb_artist_id, year, label)

                    # get artist data
                    country = 'not available'
                    city = 'not available'
                    genre = 'not available'
                    lastfm_url = f"https://www.last.fm/music/{artist_name}"
                    spotify_url = 'not available'
                    artist_check = check_artist_exists_name(cursor, artist_name)
                    
                    if artist_check: #and artist_check[0] is not None:
                        continue
                    else:
                        # update artists table
                        update_artists_table(cursor, artist_name, mb_artist_id, country, city, genre, lastfm_url, spotify_url)

            conn.commit()
            time.sleep(delay)
        conn.close()
    finally:
        conn.close()


In [39]:
# process albums batch.
# got some errors here about the label but I think it was about request limits?
process_batch(save_dir, batch_size=50,delay=1)

Row 3361: Paul Weller - Heavy Soul (1997)
Row 3362: Paul Weller - Modern Classics (1998)
Row 3363: Paul Weller - Heliocentric (2000)
Row 3364: Paul Weller - Studio 150 (2004)
Row 3365: Paul Weller - As Is Now (2005)
Row 3366: Paul Weller - Stanley Road (Deluxe Edition) (2005)
Row 3367: Paul Weller - Catch-Flame! (2 CD) (2006)
Row 3368: Paul Weller - 22 Dreams (Deluxe Edition) (2008)
Row 3369: Paul Weller - Wake Up the Nation (2010)
Row 3370: Paul Weller - Sonik Kicks (2012)
Row 3371: Pavement - Westing By Musket and Sextant (1993)
Row 3372: Pavement - Croooked Rain, Crooked Rain (1994)
ERROR :(
Row 3373: Pavement - Wowee Zowee (1995)
Row 3374: Pavement - Brighten The Corners (1997)
Row 3375: Pavement - Terror Twilight (1999)
Row 3376: Pavement - Slanted and Enchanted: Luxe and Reduxe (Remastered 2CD) (2002)
Row 3377: Pavement - Crooked Rain, Crooked Rain: L.A.'s Desert Origins (2CD) (2004)
Row 3378: Pavement - Quarantine the Past (2010)
Row 3379: Paws - Youth Culture Forever (2014)
Row

ERROR :(
Row 3526: Project Nim - Where the Nothings Live (1997)
ERROR :(
Row 3527: Proud Mary - Ocean Park (2010)
ERROR :(
Row 3528: Psychic Stunts - Panic in Motion (2009)
Row 3529: Pugwash - Jollity (2005)
Row 3530: Pugwash - Elevent Modern Antiquities (2008)
ERROR :(
Row 3531: Pugwash - The Olympus Sound (2011)
Row 3532: Pulled Apart by Horses - Pulled Apart by Horses + Demos (2010)
Row 3533: Pull in Emergency - Pull in Emergency (2010)
Row 3534: Pull Tiger Tail - PAWS. (2009)
Row 3535: Pull Tiger Tail - The Lost World (2009)
Row 3536: Pulp - It (1983)
Row 3537: Pulp - Freaks (1986)
Row 3538: Pulp - Separations (1992)
Row 3539: Pulp - His 'N' Hers (1994)
Row 3540: Pulp - Different Class* (1995)
Row 3541: Pulp - Countdown 1992-1983 (2 CD) (1996)
Row 3542: Pulp - This Is Hardcore* (1998)
Row 3543: Pulp - We Love Life* (2001)
Row 3544: Pulp - Hits* (2002)
Row 3545: Pulp - His n' Hers [Deluxe Edition] (2006)
Row 3546: Pulp - Different Class [Deluxe Edition] (2006)
Row 3547: Pulp - This 

Row 3690: Rose Melberg - Portola (1998)
Row 3691: Rose Melberg - Cast Away the Clouds (2006)
Row 3692: Rose Melberg - Homemade Ship (2009)
Row 3693: Rose Melberg - September (2013)
Row 3694: Roses Kings Castles - Roses Kings Castles (2008)
Row 3695: Rotary Ten - These Are Our Hands (2008)
Row 3696: Routes - Left My Mind (2010)
Row 3697: Roxy Music - Street Life: 20 Greatest Hits (1986)
Row 3698: Royal City - At Rush Hour the Cars (2000)
Row 3699: Royal City - Alone at the Microphone (2001)
Row 3700: Royal City - Little Heart's Ease (2004)
Row 3701: Royal City - Royal City (2009)
Row 3702: RPA & The United Nations of Sound - RPA & The United Nations of Sound (2010)
ERROR :(
Row 3703: Rubin - Componé, Ladrón (2004)
Row 3704: Rufus Wainwright - Release the Stars (2007)
Row 3705: Rufus Wainwright - All Days Are Night Songs for Lulu (2010)
Row 3706: Russian Red - I Love Your Glasses (2008)
Row 3707: Russian Red - Fuerteventura (2011)
Row 3708: Russian Red - Agent Cooper (2014)
Row 3709: Rya

Row 3858: Sibiria - Inom Familjen (2006)
Row 3859: Sibrydion - Campfire Classics (2009)
Row 3860: Siri Nilsen - Alle Snakker Sant (2011)
Row 3861: Sister Vanilla - Little Pop Rock (2007)
Row 3862: Silver Jews - Starlite Walker (1994)
Row 3863: Silver Jews - The Natural Bridge (1996)
Row 3864: Silver Jews - American Water (1998)
Row 3865: Silver Jews - Bright Flight (2001)
Row 3866: Silver Jews - Tanglewood Numbers (2005)
Row 3867: Silver Jews - Lookout Mountain, Lookout Sea (2008)
Row 3868: Silver Jews - Early Times (2012)
Row 3869: Silver Sun - Silver Sun [UK] (1997)
Row 3870: Silver Sun - 'B' Is for Silver Sun (1997)
Row 3871: Silver Sun - You Are Here (Japan) (1997)
Row 3872: Silver Sun - Neo Wave (1998)
Row 3873: Silver Sun - Disappear Here (2004)
Row 3874: Silver Sun - Dad's Weird Dream (2006)
Row 3875: Silversun Pickups - Carnavas (2006)
Row 3876: Silversun PIckups - Swoon (2009)
Row 3877: Silversun Pickups - Neck of the Woods (2012)
Row 3878: Simple Minds - Sons And Fascination 

Row 4022: Spacemen 3 - Performance (1988)
Row 4023: Spacemen 3 - Playing With Fire (1989)
Row 4024: Spacemen 3 - Recurring (1991)
Row 4025: Sparklehorse - Vivadixiesubmarinetransmissionplot (1995)
Row 4026: Sparklehorse - Good Morning Spider (1998)
Row 4027: Sparklehorse - It's A Wonderful Life (2001)
Row 4028: Sparklehorse - Dreamt For Light (2006)
Row 4029: Sparkle*jets u.k. - Bamboo Lounge (2001)
ERROR :(
Row 4030: SPC ECO - 3-D (2009)
Row 4031: SPC ECO - You Tell Me (2011)
Row 4032: SPC ECO - Dark Notes (2012)
Row 4033: SPC ECO - Sirens and Satellites (2013)
Row 4034: Spearmint - Songs for the Colour Yellow (1998)
Row 4035: Spearmint - A Week Away (1999)
Row 4036: Spearmint - Oklahoma (2000)
Row 4037: Spearmint - A Different Lifetime (2001)
Row 4038: Spearmint - My Missing Days (2003)
Row 4039: Spearmint - A Leopard and Other Stories (2004)
Row 4040: Spearmint - The Boy and the Girl That Got Away (2005)
Row 4041: Spearmint - Paris in a Bottle (2006)
Row 4042: Speedmarket Avenue - I

...

Row 5580: The Submarines - Love Notes/LetteR bombs (2011)
Row 5581: The Subways - Young for Eternity (2005)
Row 5582: The Subways - All or Nothing (2008)
Row 5583: The Subways - Money and Celebrity (2011)
Row 5584: The Sugargliders - We're All Trying to Get There (1993)
Row 5585: The Sugargliders - A Nest With a View (2012)
Row 5586: The Sugarplastic - Resin (2000)
Row 5587: The Sugars - The Curse of the Sugars (2008)
Row 5588: The Sunday Drivers - Little Heart Attacks (2004)
Row 5589: The Sunday Drivers - Tiny Telephone (2007)
Row 5590: The Sunday Drivers - The End of Maiden Trip (2009)
Row 5591: The Sunny Street - They Hurt You Everyday (2007)
Row 5592: The Sunshine Underground - Raise the Alarm (2006)
Row 5593: The Supernaturals - It Doesn't Matter Anymore (1997)
Row 5594: The Supernaturals - A Tune a Day (1998)
Row 5595: The Supernaturals - What We Did Last Summer (2002)
Row 5596: The Swirlies - What To Do About Them (1992)
Row 5597: The Tables - Shady Whims & Obstacles (1992)
Row 

ERROR :(
Row 5732: The Wave Pictures - If You Leave It Alone (2009)
Row 5733: The Wave Pictures - Play Some Pool (2009)
Row 5734: The Wave Pictures - Susan Rode the Cyclone (2010)
Row 5735: The Wave Pictures - Beer in the Breakers (2CD) (2011)
Row 5736: The Wave Pictures - Long Black Cars (2012)
Row 5737: The Wave Pictures - The Songs of Jason Molina (2013)
Row 5738: The Wave Pictures - City Forgiveness (2013)
Row 5739: The Weakerthans - Left And Leaving (2000)
Row 5740: The Weakerthans - Reconstruction Site (2003)
Row 5741: The Weakerthans - Reunion Tour (2007)
Row 5742: The Webb Brothers - Maroon (2001)
Row 5743: The Wedding Present - George Best Plus (1988)
Row 5744: The Wedding Present - Tommy (1988)
Row 5745: The Wedding Present - The Hit Parade 2 (1992)
Row 5746: The Wedding Present - Watusi (1994)
Row 5747: The Wedding Present - MINI Plus (1996)
Row 5748: The Wedding Present - Saturnalia (1996)
Row 5749: The Wedding Present - Ukranian John Peel Sessions (2000)
Row 5750: The Wedd

Row 5893: Tomte - Buchstaben über der Stadt (2006)
Row 5894: Tom the Lion - The Adventures of Tom the Lion (2011)
Row 5895: Tom Vek - We Have Sound (2005)
Row 5896: Tom Waits - Original Album Series Box Set (2011)
Row 5897: Tom y el Niño Elefante - Tom y el Niño Elefante (2010)
ERROR :(
Row 5898: Tom y la Bestia Bebe - Fin de Semana de Muertes (2009)
Row 5899: Tony Molina - Dissed and Dismissed (2013)
Row 5900: TOPS - Picture You Staring (2014)
Row 5901: Toploader - Onka's Big Moka (2000)
Row 5902: Tortoise & Bonnie 'Prince' Billy - The Brave and the Bold (2006)
Row 5903: Toy - TOY (2012)
Row 5904: Toy - Join the Dots (2014)
Row 5905: Trailer Trash Tracys - Ester (2012)
Row 5906: Tramway - A Brand of Lovin' (1991)
Row 5907: Transatlantic - Bridge Across Forever (2002)
Row 5908: Transmittens - Our Dreams (2009)
Row 5909: Transmittens - We Disappear (2010)
Row 5910: Travis - Good Feeling (1997)
Row 5911: Travis - The Man Who* (1999)
Row 5912: Travis - The Invisible Band* (2001)
Row 5913:

Row 6057: Walker Kong - There Goes the Sun (2011)
Row 6058: Walker Kong - Transparent Life (2004)
Row 6059: Walker Kong - Deliver Us From People (2007)
Row 6060: Wanderlust - Prize (1995)
Row 6061: Wanderlust - Wanderlust (1998)
Row 6062: Wanderlust - Lust and Found (2004)
Row 6063: Warm in the Wake - Night Wounds (2012)
ERROR :(
Row 6064: Warpaint - Warpaint (2014)
Row 6065: Warsaw - Warsaw (The Lost Album) (2008)
Row 6066: Waterdeep - Pink & Blue (2CD) (2008)
Row 6067: Wavves - Wavves (2008)
Row 6068: Wavves - King of the Beach (2010)
Row 6069: Wavves - Afraid of Heights (2013)
Row 6070: Waxahatchee - American Weekend (2012)
Row 6071: Waxahatchee - Cerulean Salt (2013)
Row 6072: Weakness - Weakness (2009)
Row 6073: We All Have Hooks For Hands - The Pretender (2007)
Row 6074: We Are Scientists - Safety, Fun, And Learning (In That Order) (2002)
Row 6075: We Are Scientists - With Love And Squalor (2005)
Row 6076: We Are Standard - We Are Standard (2008)
Row 6077: We Are Scientists - Bar

Row 6230: Yuck - Yuck (2011)
Row 6231: Yuck - Yuck [Deluxe] (2011)
Row 6232: Yuck - Glow & Behold (2013)
Row 6233: Zoey Van Goey - The Cage Was Unlocked All Along (2009)
Row 6234: Zoey Van Goey - Propeller Versus Wings (2011)
Row 6235: Zombies of the Stratosphere - The Well-Mannered Look (2007)
Row 6236: Zumpano - Look What the Rookie Did (1994)
Row 6237: Zumpano - Goin' Through Changes (1996)
